### Constraints on a Delta table

https://docs.databricks.com/en/tables/constraints.html

Two types of constraints are supported:

- NOT NULL: indicates that values in specific columns cannot be null.
- CHECK: indicates that a specified Boolean expression must be true for each input row.

These are the constraints we can now explore.

##### Notes about primary and foreign keys

Primary key and foreign key constraints are available in Databricks Runtime 11.1 and above.

Primary key and foreign key constraints require Unity Catalog and Delta Lake.

*You can use primary key and foreign key relationships on fields in Unity Catalog tables. Primary and foreign keys are informational only and are not enforced. Foreign keys must reference a primary key in another table.*

In [0]:
%sql

SHOW TABLES in hive_metastore.default

database,tableName,isTemporary
default,employees,false


In [0]:
%sql

SELECT * FROM hive_metastore.default.employees

id,empname,phone,email,city
1,Michelle,555-345-2222,a@loonycorn.com,King of Prussia
2,Ben,555-231-4521,b@loonycorn.com,Bloomington
3,Kristoff,555-124-2402,c@loonycorn.com,Santa Fe
4,Fatima,555-250-2386,d@loonycorn.com,Dayton
5,Eduardo,555-345-2699,e@loonycorn.com,Biloxi
6,Amy,555-001-6723,f@loonycorn.com,Spokane


In [0]:
%fs ls dbfs:/user/hive/warehouse/employees

path,name,size,modificationTime
dbfs:/user/hive/warehouse/employees/_delta_log/,_delta_log/,0,1730088121000
dbfs:/user/hive/warehouse/employees/deletion_vector_21aa09de-b9f5-4dc7-844f-c077a4267adb.bin,deletion_vector_21aa09de-b9f5-4dc7-844f-c077a4267adb.bin,43,1730089680000
dbfs:/user/hive/warehouse/employees/part-00000-d99799b3-bd02-49fd-a776-05c03969617a.c000.snappy.parquet,part-00000-d99799b3-bd02-49fd-a776-05c03969617a.c000.snappy.parquet,1460,1730088123000
dbfs:/user/hive/warehouse/employees/part-00000-ddc11eea-9cf4-429a-aaea-62a0bd03eb82.c000.snappy.parquet,part-00000-ddc11eea-9cf4-429a-aaea-62a0bd03eb82.c000.snappy.parquet,1540,1730089681000
dbfs:/user/hive/warehouse/employees/part-00000-e5d3b3de-5b70-4a6e-be57-57e17df5ae13.c000.snappy.parquet,part-00000-e5d3b3de-5b70-4a6e-be57-57e17df5ae13.c000.snappy.parquet,1208,1730088785000
dbfs:/user/hive/warehouse/employees/part-00000-f42db0e9-caca-43b6-9882-09805186985d.c000.snappy.parquet,part-00000-f42db0e9-caca-43b6-9882-09805186985d.c000.snappy.parquet,1734,1730089688000


#### Alter the Delta table

In [0]:
%sql

ALTER TABLE hive_metastore.default.employees
ADD COLUMN birthdate TIMESTAMP

#### Set a CHECK constraint on the new column
These can be added after the table has been created by means of an ALTER statement.

Here, the birthdate can be null or after 1-Jan-1900.

In [0]:
%sql

ALTER TABLE hive_metastore.default.employees
ADD CONSTRAINT validbirthdate 
  CHECK (birthdate IS NULL OR birthdate >= '1900-01-01')

#### Find the constraint definition under the "properties" column

In [0]:
%sql

DESCRIBE DETAIL hive_metastore.default.employees

format,id,name,description,location,createdAt,lastModified,partitionColumns,clusteringColumns,numFiles,sizeInBytes,properties,minReaderVersion,minWriterVersion,tableFeatures,statistics
delta,770a90a8-ef34-4ede-a2d1-4eddc26bb710,hive_metastore.default.employees,Created by the file upload UI,dbfs:/user/hive/warehouse/employees,2024-10-28T04:02:01.411Z,2024-10-28T04:27:37Z,List(),List(),2,2668,"Map(delta.enableDeletionVectors -> true, delta.constraints.validbirthdate -> birthdate IS NULL OR birthdate >= '1900-01-01')",3,7,"List(checkConstraints, deletionVectors)","Map(numRowsDeletedByDeletionVectors -> 0, numDeletionVectors -> 0)"


#### Perform an update which satisfies the constraint

In [0]:
%sql

UPDATE hive_metastore.default.employees
SET birthdate = '1991-04-21'
WHERE id = 1

num_affected_rows
1


In [0]:
%sql

SELECT * from hive_metastore.default.employees

id,empname,phone,email,city,birthdate
1,Michelle,555-345-2222,a@loonycorn.com,King of Prussia,1991-04-21T00:00:00Z
2,Ben,555-231-4521,b@loonycorn.com,Bloomington,null
3,Kristoff,555-124-2402,c@loonycorn.com,Santa Fe,null
4,Fatima,555-250-2386,d@loonycorn.com,Dayton,null
5,Eduardo,555-345-2699,e@loonycorn.com,Biloxi,null
6,Amy,555-001-6723,f@loonycorn.com,Spokane,null


#### View the underlying files for the table
Given we have altered the table, added a constraint, and updated a row, new .parquet files may be present.

##### Deletion vectors for a table

Deletion vectors are a storage optimization feature that can be enabled on Delta Lake tables. By default, when a single row in a data file is deleted, the entire Parquet file containing the record must be rewritten. With deletion vectors enabled for the table, DELETE, UPDATE, and MERGE operations use deletion vectors to mark existing rows as removed or changed without rewriting the Parquet file. Subsequent reads on the table resolve current table state by applying the deletions noted by deletion vectors to the most recent table version.

In [0]:
%fs ls dbfs:/user/hive/warehouse/employees

path,name,size,modificationTime
dbfs:/user/hive/warehouse/employees/_delta_log/,_delta_log/,0,1730088121000
dbfs:/user/hive/warehouse/employees/deletion_vector_21aa09de-b9f5-4dc7-844f-c077a4267adb.bin,deletion_vector_21aa09de-b9f5-4dc7-844f-c077a4267adb.bin,43,1730089680000
dbfs:/user/hive/warehouse/employees/deletion_vector_2c3c21e1-b0dc-446b-afe2-fcca8ac5d393.bin,deletion_vector_2c3c21e1-b0dc-446b-afe2-fcca8ac5d393.bin,43,1730089783000
dbfs:/user/hive/warehouse/employees/part-00000-cc1e23d8-7b7f-4e6f-b887-0681e70efb36.c000.snappy.parquet,part-00000-cc1e23d8-7b7f-4e6f-b887-0681e70efb36.c000.snappy.parquet,1729,1730089788000
dbfs:/user/hive/warehouse/employees/part-00000-d374f4d8-fe3e-4b84-aa51-8c621c470514.c000.snappy.parquet,part-00000-d374f4d8-fe3e-4b84-aa51-8c621c470514.c000.snappy.parquet,1540,1730089783000
dbfs:/user/hive/warehouse/employees/part-00000-d99799b3-bd02-49fd-a776-05c03969617a.c000.snappy.parquet,part-00000-d99799b3-bd02-49fd-a776-05c03969617a.c000.snappy.parquet,1460,1730088123000
dbfs:/user/hive/warehouse/employees/part-00000-ddc11eea-9cf4-429a-aaea-62a0bd03eb82.c000.snappy.parquet,part-00000-ddc11eea-9cf4-429a-aaea-62a0bd03eb82.c000.snappy.parquet,1540,1730089681000
dbfs:/user/hive/warehouse/employees/part-00000-e5d3b3de-5b70-4a6e-be57-57e17df5ae13.c000.snappy.parquet,part-00000-e5d3b3de-5b70-4a6e-be57-57e17df5ae13.c000.snappy.parquet,1208,1730088785000
dbfs:/user/hive/warehouse/employees/part-00000-f42db0e9-caca-43b6-9882-09805186985d.c000.snappy.parquet,part-00000-f42db0e9-caca-43b6-9882-09805186985d.c000.snappy.parquet,1734,1730089688000


#### Confirm the entries in the transaction history
Every detail is recorded here

In [0]:
%sql

DESCRIBE HISTORY hive_metastore.default.employees

version,timestamp,userId,userName,operation,operationParameters,job,notebook,clusterId,readVersion,isolationLevel,isBlindAppend,operationMetrics,userMetadata,engineInfo
7,2024-10-28T04:29:48Z,599420620358905,contact@loonycorn.com,OPTIMIZE,"Map(predicate -> [], auto -> true, clusterBy -> [], zOrderBy -> [], batchId -> 0)",null,List(3951670535128497),1028-032811-163l393x,6,SnapshotIsolation,false,"Map(numRemovedFiles -> 2, numRemovedBytes -> 3274, p25FileSize -> 1729, numDeletionVectorsRemoved -> 1, minFileSize -> 1729, numAddedFiles -> 1, maxFileSize -> 1729, p75FileSize -> 1729, p50FileSize -> 1729, numAddedBytes -> 1729)",null,Databricks-Runtime/15.4.x-photon-scala2.12
6,2024-10-28T04:29:44Z,599420620358905,contact@loonycorn.com,UPDATE,"Map(predicate -> [""(id#6356L = 1)""])",null,List(3951670535128497),1028-032811-163l393x,5,WriteSerializable,false,"Map(numRemovedFiles -> 0, numRemovedBytes -> 0, numCopiedRows -> 0, numDeletionVectorsAdded -> 1, numDeletionVectorsRemoved -> 0, numAddedChangeFiles -> 0, executionTimeMs -> 2749, numDeletionVectorsUpdated -> 0, scanTimeMs -> 1049, numAddedFiles -> 1, numUpdatedRows -> 1, numAddedBytes -> 1540, rewriteTimeMs -> 1700)",null,Databricks-Runtime/15.4.x-photon-scala2.12
5,2024-10-28T04:28:09Z,599420620358905,contact@loonycorn.com,OPTIMIZE,"Map(predicate -> [], auto -> true, clusterBy -> [], zOrderBy -> [], batchId -> 0)",null,List(3951670535128497),1028-032811-163l393x,4,SnapshotIsolation,false,"Map(numRemovedFiles -> 3, numRemovedBytes -> 4208, p25FileSize -> 1734, numDeletionVectorsRemoved -> 1, minFileSize -> 1734, numAddedFiles -> 1, maxFileSize -> 1734, p75FileSize -> 1734, p50FileSize -> 1734, numAddedBytes -> 1734)",null,Databricks-Runtime/15.4.x-photon-scala2.12
4,2024-10-28T04:28:04Z,599420620358905,contact@loonycorn.com,UPDATE,"Map(predicate -> [""(id#5107L = 1)""])",null,List(3951670535128497),1028-032811-163l393x,3,WriteSerializable,false,"Map(numRemovedFiles -> 0, numRemovedBytes -> 0, numCopiedRows -> 0, numDeletionVectorsAdded -> 1, numDeletionVectorsRemoved -> 0, numAddedChangeFiles -> 0, executionTimeMs -> 5550, numDeletionVectorsUpdated -> 0, scanTimeMs -> 2752, numAddedFiles -> 1, numUpdatedRows -> 1, numAddedBytes -> 1540, rewriteTimeMs -> 2744)",null,Databricks-Runtime/15.4.x-photon-scala2.12
3,2024-10-28T04:27:37Z,599420620358905,contact@loonycorn.com,ADD CONSTRAINT,"Map(name -> validbirthdate, expr -> birthdate IS NULL OR birthdate >= '1900-01-01')",null,List(3951670535128497),1028-032811-163l393x,2,WriteSerializable,false,Map(),null,Databricks-Runtime/15.4.x-photon-scala2.12
2,2024-10-28T04:27:30Z,599420620358905,contact@loonycorn.com,ADD COLUMNS,"Map(columns -> [{""column"":{""name"":""birthdate"",""type"":""timestamp"",""nullable"":true,""metadata"":{}}}])",null,List(3951670535128497),1028-032811-163l393x,1,WriteSerializable,true,Map(),null,Databricks-Runtime/15.4.x-photon-scala2.12
1,2024-10-28T04:13:05Z,599420620358905,contact@loonycorn.com,WRITE,"Map(mode -> Append, statsOnLoad -> false, partitionBy -> [])",null,List(3951670535128475),1028-032811-163l393x,0,WriteSerializable,true,"Map(numFiles -> 1, numOutputRows -> 1, numOutputBytes -> 1208)",null,Databricks-Runtime/15.4.x-photon-scala2.12
0,2024-10-28T04:02:03Z,599420620358905,contact@loonycorn.com,CREATE TABLE AS SELECT,"Map(partitionBy -> [], clusterBy -> [], description -> Created by the file upload UI, isManaged -> true, properties -> {""delta.enableDeletionVectors"":""true""}, statsOnLoad -> false)",null,null,1028-032811-163l393x,null,WriteSerializable,true,"Map(numFiles -> 1, numOutputRows -> 5, numOutputBytes -> 1460)",null,Databricks-Runtime/15.4.x-photon-scala2.12


#### Run a query to violate the constraint

In [0]:
%sql

UPDATE hive_metastore.default.employees
SET birthdate = '1891-04-21'
WHERE id = 2

com.databricks.sql.transaction.tahoe.schema.DeltaInvariantViolationException: [DELTA_VIOLATE_CONSTRAINT_WITH_VALUES] CHECK constraint validbirthdate ((birthdate IS NULL) OR (birthdate >= '1900-01-01')) violated by row with values:
 - birthdate : 1891-04-21 00:00:00
	at com.databricks.sql.transaction.tahoe.schema.DeltaInvariantViolationException$.getConstraintViolationWithValuesException(InvariantViolationException.scala:82)
	at com.databricks.photon.PhotonException$.getSparkException(PhotonException.scala:279)
	at com.databricks.photon.PhotonWriteResultHandler.getResult(PhotonWriteStageExec.scala:138)
	at com.databricks.photon.PhotonBasicEvaluatorFactory$PhotonBasicEvaluator$$anon$1.hasNext(PhotonBasicEvaluatorFactory.scala:224)
	at com.databricks.photon.CloseableIterator$$anon$10.hasNext(CloseableIterator.scala:211)
	at org.apache.spark.sql.execution.datasources.FileFormatWriter$.$anonfun$writeUsingPhoton$2(FileFormatWriter.scala:645)
	at org.apache.spark.scheduler.ResultTask.$anonfun

#### Constraints can be removed by referencing their names

In [0]:
%sql

ALTER TABLE hive_metastore.default.employees
DROP CONSTRAINT validbirthdate

#### The properties no longer reflect the constraint

In [0]:
%sql

DESCRIBE DETAIL hive_metastore.default.employees

format,id,name,description,location,createdAt,lastModified,partitionColumns,clusteringColumns,numFiles,sizeInBytes,properties,minReaderVersion,minWriterVersion,tableFeatures,statistics
delta,770a90a8-ef34-4ede-a2d1-4eddc26bb710,hive_metastore.default.employees,Created by the file upload UI,dbfs:/user/hive/warehouse/employees,2024-10-28T04:02:01.411Z,2024-10-28T04:32:36Z,List(),List(),1,1729,Map(delta.enableDeletionVectors -> true),3,7,"List(checkConstraints, deletionVectors)","Map(numRowsDeletedByDeletionVectors -> 0, numDeletionVectors -> 0)"


#### Try to re-do the failed insert
It goes through this time, thanks to the absence of the constraint

In [0]:
%sql

UPDATE hive_metastore.default.employees
SET birthdate = '1891-04-21'
WHERE id = 2

num_affected_rows
1


In [0]:
%sql

SELECT * FROM hive_metastore.default.employees

id,empname,phone,email,city,birthdate
2,Ben,555-231-4521,b@loonycorn.com,Bloomington,1891-04-21T00:00:00Z
1,Michelle,555-345-2222,a@loonycorn.com,King of Prussia,1991-04-21T00:00:00Z
3,Kristoff,555-124-2402,c@loonycorn.com,Santa Fe,null
4,Fatima,555-250-2386,d@loonycorn.com,Dayton,null
5,Eduardo,555-345-2699,e@loonycorn.com,Biloxi,null
6,Amy,555-001-6723,f@loonycorn.com,Spokane,null


#### The history simply adds on to the changes recorded
Dropping a constraint that was previously added, does not undo the adding of the constraint - the history ensures that we can roll back to the state where the constraint was present

In [0]:
%sql

DESCRIBE HISTORY hive_metastore.default.employees

version,timestamp,userId,userName,operation,operationParameters,job,notebook,clusterId,readVersion,isolationLevel,isBlindAppend,operationMetrics,userMetadata,engineInfo
10,2024-10-28T04:33:20Z,599420620358905,contact@loonycorn.com,OPTIMIZE,"Map(predicate -> [], auto -> true, clusterBy -> [], zOrderBy -> [], batchId -> 0)",null,List(3951670535128497),1028-032811-163l393x,9,SnapshotIsolation,false,"Map(numRemovedFiles -> 2, numRemovedBytes -> 3222, p25FileSize -> 1738, numDeletionVectorsRemoved -> 1, minFileSize -> 1738, numAddedFiles -> 1, maxFileSize -> 1738, p75FileSize -> 1738, p50FileSize -> 1738, numAddedBytes -> 1738)",null,Databricks-Runtime/15.4.x-photon-scala2.12
9,2024-10-28T04:33:17Z,599420620358905,contact@loonycorn.com,UPDATE,"Map(predicate -> [""(id#8493L = 2)""])",null,List(3951670535128497),1028-032811-163l393x,8,WriteSerializable,false,"Map(numRemovedFiles -> 0, numRemovedBytes -> 0, numCopiedRows -> 0, numDeletionVectorsAdded -> 1, numDeletionVectorsRemoved -> 0, numAddedChangeFiles -> 0, executionTimeMs -> 2273, numDeletionVectorsUpdated -> 0, scanTimeMs -> 949, numAddedFiles -> 1, numUpdatedRows -> 1, numAddedBytes -> 1493, rewriteTimeMs -> 1322)",null,Databricks-Runtime/15.4.x-photon-scala2.12
8,2024-10-28T04:32:36Z,599420620358905,contact@loonycorn.com,DROP CONSTRAINT,"Map(name -> validbirthdate, expr -> birthdate IS NULL OR birthdate >= '1900-01-01', existed -> true)",null,List(3951670535128497),1028-032811-163l393x,7,WriteSerializable,true,Map(),null,Databricks-Runtime/15.4.x-photon-scala2.12
7,2024-10-28T04:29:48Z,599420620358905,contact@loonycorn.com,OPTIMIZE,"Map(predicate -> [], auto -> true, clusterBy -> [], zOrderBy -> [], batchId -> 0)",null,List(3951670535128497),1028-032811-163l393x,6,SnapshotIsolation,false,"Map(numRemovedFiles -> 2, numRemovedBytes -> 3274, p25FileSize -> 1729, numDeletionVectorsRemoved -> 1, minFileSize -> 1729, numAddedFiles -> 1, maxFileSize -> 1729, p75FileSize -> 1729, p50FileSize -> 1729, numAddedBytes -> 1729)",null,Databricks-Runtime/15.4.x-photon-scala2.12
6,2024-10-28T04:29:44Z,599420620358905,contact@loonycorn.com,UPDATE,"Map(predicate -> [""(id#6356L = 1)""])",null,List(3951670535128497),1028-032811-163l393x,5,WriteSerializable,false,"Map(numRemovedFiles -> 0, numRemovedBytes -> 0, numCopiedRows -> 0, numDeletionVectorsAdded -> 1, numDeletionVectorsRemoved -> 0, numAddedChangeFiles -> 0, executionTimeMs -> 2749, numDeletionVectorsUpdated -> 0, scanTimeMs -> 1049, numAddedFiles -> 1, numUpdatedRows -> 1, numAddedBytes -> 1540, rewriteTimeMs -> 1700)",null,Databricks-Runtime/15.4.x-photon-scala2.12
5,2024-10-28T04:28:09Z,599420620358905,contact@loonycorn.com,OPTIMIZE,"Map(predicate -> [], auto -> true, clusterBy -> [], zOrderBy -> [], batchId -> 0)",null,List(3951670535128497),1028-032811-163l393x,4,SnapshotIsolation,false,"Map(numRemovedFiles -> 3, numRemovedBytes -> 4208, p25FileSize -> 1734, numDeletionVectorsRemoved -> 1, minFileSize -> 1734, numAddedFiles -> 1, maxFileSize -> 1734, p75FileSize -> 1734, p50FileSize -> 1734, numAddedBytes -> 1734)",null,Databricks-Runtime/15.4.x-photon-scala2.12
4,2024-10-28T04:28:04Z,599420620358905,contact@loonycorn.com,UPDATE,"Map(predicate -> [""(id#5107L = 1)""])",null,List(3951670535128497),1028-032811-163l393x,3,WriteSerializable,false,"Map(numRemovedFiles -> 0, numRemovedBytes -> 0, numCopiedRows -> 0, numDeletionVectorsAdded -> 1, numDeletionVectorsRemoved -> 0, numAddedChangeFiles -> 0, executionTimeMs -> 5550, numDeletionVectorsUpdated -> 0, scanTimeMs -> 2752, numAddedFiles -> 1, numUpdatedRows -> 1, numAddedBytes -> 1540, rewriteTimeMs -> 2744)",null,Databricks-Runtime/15.4.x-photon-scala2.12
3,2024-10-28T04:27:37Z,599420620358905,contact@loonycorn.com,ADD CONSTRAINT,"Map(name -> validbirthdate, expr -> birthdate IS NULL OR birthdate >= '1900-01-01')",null,List(3951670535128497),1028-032811-163l393x,2,WriteSerializable,false,Map(),null,Databricks-Runtime/15.4.x-photon-scala2.12
2,2024-10-28T04:27:30Z,599420620358905,c

#### (Optional) Explore the NOT NULL constraint

In [0]:
%sql

ALTER TABLE hive_metastore.default.employees
CHANGE COLUMN empname SET NOT NULL

#### This confirms that empname cannot be NULL

In [0]:
%sql

INSERT INTO hive_metastore.default.employees (id, empname, email, city, birthdate, phone)
VALUES (7, NULL, 'g@loonycorn.com', 'Houston', '1983-11-07', NULL)

com.databricks.sql.transaction.tahoe.schema.DeltaInvariantViolationException: [DELTA_NOT_NULL_CONSTRAINT_VIOLATED] NOT NULL constraint violated for column: empname.

	at com.databricks.sql.transaction.tahoe.schema.DeltaInvariantViolationException$.getNotNullInvariantViolationException(InvariantViolationException.scala:57)
	at com.databricks.photon.PhotonException$.getSparkException(PhotonException.scala:268)
	at com.databricks.photon.PhotonWriteResultHandler.getResult(PhotonWriteStageExec.scala:138)
	at com.databricks.photon.PhotonBasicEvaluatorFactory$PhotonBasicEvaluator$$anon$1.hasNext(PhotonBasicEvaluatorFactory.scala:224)
	at com.databricks.photon.CloseableIterator$$anon$10.hasNext(CloseableIterator.scala:211)
	at org.apache.spark.sql.execution.datasources.FileFormatWriter$.$anonfun$writeUsingPhoton$2(FileFormatWriter.scala:645)
	at org.apache.spark.scheduler.ResultTask.$anonfun$runTask$3(ResultTask.scala:82)
	at com.databricks.spark.util.ExecutorFrameProfiler$.record(ExecutorFram

In [0]:
%sql

INSERT INTO hive_metastore.default.employees (id, empname, email, city, birthdate, phone)
VALUES (7, 'Ingrid', 'g@loonycorn.com', 'Houston', '1983-11-07', NULL)

num_affected_rows,num_inserted_rows
1,1


In [0]:
%sql

SELECT * FROM hive_metastore.default.employees
ORDER BY id

id,empname,phone,email,city,birthdate
1,Michelle,555-345-2222,a@loonycorn.com,King of Prussia,1991-04-21T00:00:00Z
2,Ben,555-231-4521,b@loonycorn.com,Bloomington,1891-04-21T00:00:00Z
3,Kristoff,555-124-2402,c@loonycorn.com,Santa Fe,null
4,Fatima,555-250-2386,d@loonycorn.com,Dayton,null
5,Eduardo,555-345-2699,e@loonycorn.com,Biloxi,null
6,Amy,555-001-6723,f@loonycorn.com,Spokane,null
7,Ingrid,null,g@loonycorn.com,Houston,1983-11-07T00:00:00Z
